In [2]:
import gmaps
import pandas as pd
import requests
from config import g_key
import matplotlib.pyplot as plt

In [4]:
weather_data=pd.read_csv("Data/weatherpy_challenge.csv")
gmaps.configure(api_key=g_key)

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
mintemp = float(input("What is the minimum temperature you would like for your trip? "))
maxtemp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 5
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) n
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_temp_df = weather_data.loc[(weather_data["Max Temp"] <= maxtemp) & (weather_data["Max Temp"] >= mintemp)]
if rain == 'yes':
    preferred_precipitation= preferred_temp_df.loc[(preferred_temp_df['Rainfall']>0.)]
else:
    preferred_precipitation= preferred_temp_df.loc[(preferred_temp_df['Rainfall']==0.)]
if snow == 'yes':
    preferred_cities=preferred_precipitation.loc[preferred_precipitation['Snowfall']>0.]
else:
    preferred_cities=preferred_precipitation.loc[preferred_precipitation['Snowfall']==0.]

In [7]:
len(preferred_cities)

490

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Current Conditions", "Lat", "Lng","Max Temp"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Conditions,Lat,Lng,Max Temp,Hotel Name
0,Cape Town,ZA,broken clouds,-33.93,18.42,55.99,
2,Ushuaia,AR,broken clouds,-54.80,-68.30,50.00,
3,Hamirpur,IN,clear sky,25.95,80.15,75.90,
4,Port Alfred,ZA,scattered clouds,-33.59,26.89,59.83,
5,Jalu,LY,clear sky,29.03,21.55,56.80,
6,Grasse,FR,clear sky,43.67,6.92,59.00,
7,Sept-Iles,CA,broken clouds,50.20,-66.38,41.00,
8,Longyearbyen,SJ,broken clouds,78.22,15.64,14.00,
9,Plettenberg Bay,ZA,broken clouds,-34.05,23.37,54.93,
11,Jamestown,US,broken clouds,42.10,-79.24,52.00,


In [10]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": " "}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print ('Found')
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
hotel_df.to_csv("Data/WeatherPy_vacation.csv", index_label="City_ID")

In [12]:
info_box_template = '''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Conditions} and {Max Temp} °F</dd>
</dl>
'''
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:

# Add marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))